In [1]:
from QueryParser import QueryParser

In [2]:
sql = f"""WITH recent AS (
    SELECT account_id, balance
    FROM core.accounts
    WHERE updated_at >= DATEADD('day', -7, CURRENT_DATE)
)
SELECT
    r.account_id,
    b.balance_bucket
FROM recent r
JOIN analytics.balance_lookup b
    ON r.balance + 10 = b.bucket_start
"""
parser_a = QueryParser(sql)

In [3]:
parser_a.filters()

[{'query': 'updated_at >= DATEADD(DAY, -7, CURRENT_DATE)',
  'filter_type': 'WHERE',
  'operator': '>=',
  'columns': [Column(name='updated_at', potential_tables=['core.accounts'], lineage=None)]}]

In [4]:
parser_a.column_lineage()

{'recent': {'account_id': ['core.accounts'], 'balance': ['core.accounts']}}

In [5]:
parser_a.feature_columns()

[{'name': 'account_id', 'potential_tables': ['core.accounts']},
 {'name': 'balance_bucket', 'potential_tables': ['analytics.balance_lookup']},
 {'name': 'bucket_start', 'potential_tables': ['analytics.balance_lookup']},
 {'name': 'balance', 'potential_tables': ['core.accounts']},
 {'name': 'updated_at', 'potential_tables': ['core.accounts']}]

In [6]:
parser_a.joins()

[{'join_type': 'INNER JOIN',
  'column_left': Column(name='balance', potential_tables=['core.accounts'], lineage=None),
  'column_right': Column(name='bucket_start', potential_tables=['analytics.balance_lookup'], lineage=None),
  'complex_left': 'r.balance + 10'}]

In [7]:
sql = """
SELECT
    a.COLUMN_A,
    b.COLUMN_B,
    AMBIGUOUS_COL
FROM DB.SCHEMA.TABLE_A a
JOIN DB.SCHEMA.TABLE_B b
    ON a.COLUMN_A = b.COLUMN_B
"""
parser_b = QueryParser(sql)

In [8]:
parser_b.feature_columns()

[{'name': 'COLUMN_A', 'potential_tables': ['DB.SCHEMA.TABLE_A']},
 {'name': 'COLUMN_B', 'potential_tables': ['DB.SCHEMA.TABLE_B']},
 {'name': 'AMBIGUOUS_COL',
  'potential_tables': ['DB.SCHEMA.TABLE_A', 'DB.SCHEMA.TABLE_B']}]

In [9]:
parser_b.joins()

[{'join_type': 'INNER JOIN',
  'column_left': Column(name='COLUMN_A', potential_tables=['DB.SCHEMA.TABLE_A'], lineage=None),
  'column_right': Column(name='COLUMN_B', potential_tables=['DB.SCHEMA.TABLE_B'], lineage=None)}]

In [10]:
sql = f"""
SELECT
    SUMMED_FIELD
FROM
    (SELECT
        a.A + b.B AS SUMMED_FIELD
    FROM
        MYDB.MYSCHEMA.TABLE_A a
    JOIN
        MYDB.MYSCHEMA.TABLE_B b
    ON a.ID = b.ID
    )
WHERE
    SUMMED_FIELD > 10;
"""
parser_c = QueryParser(sql)

parser_c.source_columns()

[Column(name='A', potential_tables=['MYDB.MYSCHEMA.TABLE_A'], lineage=None),
 Column(name='B', potential_tables=['MYDB.MYSCHEMA.TABLE_B'], lineage=None),
 Column(name='ID', potential_tables=['MYDB.MYSCHEMA.TABLE_A'], lineage=None),
 Column(name='ID', potential_tables=['MYDB.MYSCHEMA.TABLE_B'], lineage=None)]

In [11]:
parser_c.column_lineage()

{}

In [12]:
parser_c.filters()

[{'query': 'SUMMED_FIELD > 10',
  'filter_type': 'WHERE',
  'operator': '>',
  'columns': [Column(name='SUMMED_FIELD', potential_tables=['MYDB.MYSCHEMA.TABLE_A', 'MYDB.MYSCHEMA.TABLE_B'], lineage=[Column(name='A', potential_tables=['MYDB.MYSCHEMA.TABLE_A'], lineage=None), Column(name='B', potential_tables=['MYDB.MYSCHEMA.TABLE_B'], lineage=None), Column(name='SUMMED_FIELD', potential_tables=['MYDB.MYSCHEMA.TABLE_A', 'MYDB.MYSCHEMA.TABLE_B'], lineage=[Column(name='A', potential_tables=['MYDB.MYSCHEMA.TABLE_A'], lineage=None), Column(name='B', potential_tables=['MYDB.MYSCHEMA.TABLE_B'], lineage=None)])])]}]

In [13]:
parser_c.source_columns()

[Column(name='A', potential_tables=['MYDB.MYSCHEMA.TABLE_A'], lineage=None),
 Column(name='B', potential_tables=['MYDB.MYSCHEMA.TABLE_B'], lineage=None),
 Column(name='ID', potential_tables=['MYDB.MYSCHEMA.TABLE_A'], lineage=None),
 Column(name='ID', potential_tables=['MYDB.MYSCHEMA.TABLE_B'], lineage=None)]

In [14]:
sql = """
SELECT COUNT(DISTINCT A.CDN) FROM MYDB.MYSQL.TBL_A A
INNER JOIN MYDB.MYSQL.TBL_B B

ON A.ID = B.ID
WHERE A.END_DT = '9999-99-99' AND B.PARTY_ROLE = 'CUST';
"""
parser_d = QueryParser(sql)

parser_d.source_tables()

['MYDB.MYSQL.TBL_A', 'MYDB.MYSQL.TBL_B']

In [15]:
sql = """
SELECT
    COUNT(1)
FROM
    (SELECT
        MY_COL,
        CASE WHEN OTHER_COL = 1 THEN '1' ELSE '0' END AS ANOTHER_THING
    FROM MYDB.MYSCHEMA. MY_TABLE A
    ) A
GROUP BY 1
"""
parser_e = QueryParser(sql)

parser_e.source_columns()

[Column(name='MY_COL', potential_tables=['MYDB.MYSCHEMA.MY_TABLE'], lineage=None),
 Column(name='OTHER_COL', potential_tables=['MYDB.MYSCHEMA.MY_TABLE'], lineage=None)]

In [16]:
sql = """
SELECT
    ADDR_CNT,
    COUNT(EPI) CUST_CNT
FROM
    (
        SELECT
            EPI,
            COUNT(DISTINCT AD_DIM_NB) ADDR_CNT
        FROM
            DB.SCHEMA_V.ATABLE
        WHERE
            END_DT > dateadd (month, -6, current_date)
        GROUP BY
            1
        HAVING
            ADDR_CNT > 2
    )
GROUP BY
    1;
"""

parser_f = QueryParser(sql)

parser_f.source_columns()

[Column(name='AD_DIM_NB', potential_tables=['DB.SCHEMA_V.ATABLE'], lineage=None),
 Column(name='EPI', potential_tables=['DB.SCHEMA_V.ATABLE'], lineage=None),
 Column(name='END_DT', potential_tables=['DB.SCHEMA_V.ATABLE'], lineage=None)]

In [17]:
parser_f.select_columns()

[{'column': Column(name='AD_DIM_NB', potential_tables=['DB.SCHEMA_V.ATABLE'], lineage=None),
  'direct': False},
 {'column': Column(name='EPI', potential_tables=['DB.SCHEMA_V.ATABLE'], lineage=None),
  'direct': False}]

In [ ]:
sql = f"""
SELECT
    a.DIRECT_COL AS COLUMN_1,
    b.COLUMN_1 + 10 AS COLUMN_2,
    b.COLUMN_2 AS COLUMN_3
FROM
    TABLE_A
JOIN
    (SELECT

    )